# Preamble

In [ ]:
import tensorflow
tensorflow.compat.v1.disable_eager_execution()
print(tensorflow.__version__)

In [2]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import collections

import numpy as np

from tensorflow.python.eager import context
from tensorflow.python.framework import ops
from tensorflow.python.framework import tensor_shape
from tensorflow.python.keras import activations
from tensorflow.python.keras import backend as K
from tensorflow.python.keras import constraints
from tensorflow.python.keras import initializers
from tensorflow.python.keras import regularizers
from tensorflow.python.keras.engine.base_layer import Layer
from tensorflow.python.keras.engine.input_spec import InputSpec
from tensorflow.python.keras.utils import generic_utils
from tensorflow.python.keras.utils import tf_utils
from tensorflow.python.ops import array_ops
from tensorflow.python.ops import control_flow_util
from tensorflow.python.ops import state_ops
from tensorflow.python.platform import tf_logging as logging
from tensorflow.python.training.tracking import base as trackable
from tensorflow.python.training.tracking import data_structures
from tensorflow.python.util import nest
from tensorflow.python.util.tf_export import keras_export
from time import time

# ECNN classes

In [3]:
from tensorflow.python.keras.layers.recurrent import RNN,DropoutRNNCellMixin

## ECNNCell class

In [5]:
class ECNNCell(DropoutRNNCellMixin, Layer):
  """Cell class for ECNN.

  Arguments:
    units: Positive integer, dimensionality of the output space.
    activation: Activation function to use.
      Default: hyperbolic tangent (`tanh`).
      If you pass `None`, no activation is applied
      (ie. "linear" activation: `a(x) = x`).
    use_bias: Boolean, whether the layer uses a bias vector.
    kernel_initializer: Initializer for the `kernel` weights matrix,
      used for the linear transformation of the inputs.
    recurrent_initializer: Initializer for the `recurrent_kernel`
      weights matrix, used for the linear transformation of the recurrent state.
    bias_initializer: Initializer for the bias vector.
    kernel_regularizer: Regularizer function applied to
      the `kernel` weights matrix.
    recurrent_regularizer: Regularizer function applied to
      the `recurrent_kernel` weights matrix.
    bias_regularizer: Regularizer function applied to the bias vector.
    kernel_constraint: Constraint function applied to
      the `kernel` weights matrix.
    recurrent_constraint: Constraint function applied to
      the `recurrent_kernel` weights matrix.
    bias_constraint: Constraint function applied to the bias vector.
    dropout: Float between 0 and 1.
      Fraction of the units to drop for
      the linear transformation of the inputs.
    recurrent_dropout: Float between 0 and 1.
      Fraction of the units to drop for
      the linear transformation of the recurrent state.

  Call arguments:
    inputs: A 2D tensor.
    states: List of state tensors corresponding to the previous timestep.
    training: Python boolean indicating whether the layer should behave in
      training mode or in inference mode. Only relevant when `dropout` or
      `recurrent_dropout` is used.
  """

  def __init__(self,
               units,
               activation='tanh',
               use_bias=True,
               kernel_initializer='glorot_uniform',
               recurrent_initializer='orthogonal',
               bias_initializer='zeros',
               kernel_regularizer=None,
               recurrent_regularizer=None,
               bias_regularizer=None,
               kernel_constraint=None,
               recurrent_constraint=None,
               bias_constraint=None,
               dropout=0.,
               recurrent_dropout=0.,
               **kwargs):
    super(ECNNCell, self).__init__(**kwargs)
    self.units = units
    self.activation = activations.get(activation)
    self.use_bias = use_bias

    self.kernel_initializer = initializers.get(kernel_initializer)
    self.recurrent_initializer = initializers.get(recurrent_initializer)
    self.bias_initializer = initializers.get(bias_initializer)

    self.kernel_regularizer = regularizers.get(kernel_regularizer)
    self.recurrent_regularizer = regularizers.get(recurrent_regularizer)
    self.bias_regularizer = regularizers.get(bias_regularizer)

    self.kernel_constraint = constraints.get(kernel_constraint)
    self.recurrent_constraint = constraints.get(recurrent_constraint)
    self.bias_constraint = constraints.get(bias_constraint)

    self.dropout = min(1., max(0., dropout))
    self.recurrent_dropout = min(1., max(0., recurrent_dropout))
    self.state_size = self.units
    self.output_size = self.units

  @tf_utils.shape_type_conversion
  def build(self, input_shape):
    self.kernel = self.add_weight(
        shape=(input_shape[-1]-1, self.units),
        name='kernel',
        initializer=self.kernel_initializer,
        regularizer=self.kernel_regularizer,
        constraint=self.kernel_constraint)
    
    self.D = self.add_weight(shape=(1, self.units),
                                      name='kernel',
                                      initializer=self.kernel_initializer,
                                      regularizer=self.kernel_regularizer,
                                      constraint=self.kernel_constraint)
    self.C = self.add_weight(shape=(self.units,1),
                                      name='kernel',
                                      initializer=self.kernel_initializer,
                                      regularizer=self.kernel_regularizer,
                                      constraint=self.kernel_constraint)
    self.recurrent_kernel = self.add_weight(shape=(self.units, self.units),
                                      name='recurrent_kernel',
                                      initializer=self.recurrent_initializer,
                                      regularizer=self.recurrent_regularizer,
                                      constraint=self.recurrent_constraint)
    if self.use_bias:
      self.bias = self.add_weight(
          shape=(self.units,),
          name='bias',
          initializer=self.bias_initializer,
          regularizer=self.bias_regularizer,
          constraint=self.bias_constraint)
    else:
      self.bias = None
    self.built = True

 

  def call(self, inputs, states, training=None):
    prev_output = states[0]
    dp_mask = self.get_dropout_mask_for_cell(inputs, training)
    rec_dp_mask = self.get_recurrent_dropout_mask_for_cell(
        prev_output, training)
    obs= inputs[:,-1]
    obs = K.expand_dims(obs,axis = 1)
    if dp_mask is not None:
      h = K.dot(inputs[:,:-1]* dp_mask, self.kernel)
    else:
      h = K.dot(inputs[:,:-1], self.kernel)
    if self.bias is not None:
      h = K.bias_add(h, self.bias)

    if rec_dp_mask is not None:
      prev_output *= rec_dp_mask
    simple_rec = h + K.dot(prev_output, self.recurrent_kernel)
    Error = K.dot((K.tanh(K.dot(prev_output, self.C) - obs)),self.D)
    output = simple_rec + Error
    if self.activation is not None:
      output = self.activation(output)

    return output, [output]

  #def get_initial_state(self, inputs=None, batch_size=None, dtype=None):
  #  return _generate_zero_filled_state_for_cell(self, inputs, batch_size, dtype)

  def get_config(self):
    config = {
        'units':
            self.units,
        'activation':
            activations.serialize(self.activation),
        'use_bias':
            self.use_bias,
        'kernel_initializer':
            initializers.serialize(self.kernel_initializer),
        'recurrent_initializer':
            initializers.serialize(self.recurrent_initializer),
        'bias_initializer':
            initializers.serialize(self.bias_initializer),
        'kernel_regularizer':
            regularizers.serialize(self.kernel_regularizer),
        'recurrent_regularizer':
            regularizers.serialize(self.recurrent_regularizer),
        'bias_regularizer':
            regularizers.serialize(self.bias_regularizer),
        'kernel_constraint':
            constraints.serialize(self.kernel_constraint),
        'recurrent_constraint':
            constraints.serialize(self.recurrent_constraint),
        'bias_constraint':
            constraints.serialize(self.bias_constraint),
        'dropout':
            self.dropout,
        'recurrent_dropout':
            self.recurrent_dropout
    }
    base_config = super(ECNNCell, self).get_config()
    return dict(list(base_config.items()) + list(config.items()))


## ECNN class

In [6]:
class ECNN(RNN):
  """Fully-connected RNN where the output is to be fed back to input.

  Arguments:
    units: Positive integer, dimensionality of the output space.
    activation: Activation function to use.
      Default: hyperbolic tangent (`tanh`).
      If you pass None, no activation is applied
      (ie. "linear" activation: `a(x) = x`).
    use_bias: Boolean, whether the layer uses a bias vector.
    kernel_initializer: Initializer for the `kernel` weights matrix,
      used for the linear transformation of the inputs.
    recurrent_initializer: Initializer for the `recurrent_kernel`
      weights matrix,
      used for the linear transformation of the recurrent state.
    bias_initializer: Initializer for the bias vector.
    kernel_regularizer: Regularizer function applied to
      the `kernel` weights matrix.
    recurrent_regularizer: Regularizer function applied to
      the `recurrent_kernel` weights matrix.
    bias_regularizer: Regularizer function applied to the bias vector.
    activity_regularizer: Regularizer function applied to
      the output of the layer (its "activation")..
    kernel_constraint: Constraint function applied to
      the `kernel` weights matrix.
    recurrent_constraint: Constraint function applied to
      the `recurrent_kernel` weights matrix.
    bias_constraint: Constraint function applied to the bias vector.
    dropout: Float between 0 and 1.
      Fraction of the units to drop for
      the linear transformation of the inputs.
    recurrent_dropout: Float between 0 and 1.
      Fraction of the units to drop for
      the linear transformation of the recurrent state.
    return_sequences: Boolean. Whether to return the last output
      in the output sequence, or the full sequence.
    return_state: Boolean. Whether to return the last state
      in addition to the output.
    go_backwards: Boolean (default False).
      If True, process the input sequence backwards and return the
      reversed sequence.
    stateful: Boolean (default False). If True, the last state
      for each sample at index i in a batch will be used as initial
      state for the sample of index i in the following batch.
    unroll: Boolean (default False).
      If True, the network will be unrolled,
      else a symbolic loop will be used.
      Unrolling can speed-up a RNN,
      although it tends to be more memory-intensive.
      Unrolling is only suitable for short sequences.

  Call arguments:
    inputs: A 3D tensor.
    mask: Binary tensor of shape `(samples, timesteps)` indicating whether
      a given timestep should be masked.
    training: Python boolean indicating whether the layer should behave in
      training mode or in inference mode. This argument is passed to the cell
      when calling it. This is only relevant if `dropout` or
      `recurrent_dropout` is used.
    initial_state: List of initial state tensors to be passed to the first
      call of the cell.
  """

  def __init__(self,
               units,
               activation='tanh',
               use_bias=True,
               kernel_initializer='glorot_uniform',
               recurrent_initializer='orthogonal',
               bias_initializer='zeros',
               kernel_regularizer=None,
               recurrent_regularizer=None,
               bias_regularizer=None,
               activity_regularizer=None,
               kernel_constraint=None,
               recurrent_constraint=None,
               bias_constraint=None,
               dropout=0.,
               recurrent_dropout=0.,
               return_sequences=False,
               return_state=False,
               go_backwards=False,
               stateful=False,
               unroll=False,
               **kwargs):
    if 'implementation' in kwargs:
      kwargs.pop('implementation')
      logging.warning('The `implementation` argument '
                      'in `SimpleRNN` has been deprecated. '
                      'Please remove it from your layer call.')
    cell = ECNNCell(
        units,
        activation=activation,
        use_bias=use_bias,
        kernel_initializer=kernel_initializer,
        recurrent_initializer=recurrent_initializer,
        bias_initializer=bias_initializer,
        kernel_regularizer=kernel_regularizer,
        recurrent_regularizer=recurrent_regularizer,
        bias_regularizer=bias_regularizer,
        kernel_constraint=kernel_constraint,
        recurrent_constraint=recurrent_constraint,
        bias_constraint=bias_constraint,
        dropout=dropout,
        recurrent_dropout=recurrent_dropout)
    super(ECNN, self).__init__(
        cell,
        return_sequences=return_sequences,
        return_state=return_state,
        go_backwards=go_backwards,
        stateful=stateful,
        unroll=unroll,
        **kwargs)
    self.activity_regularizer = regularizers.get(activity_regularizer)
    self.input_spec = [InputSpec(ndim=3)]

  def call(self, inputs, mask=None, training=None, initial_state=None):
    self.cell.reset_dropout_mask()
    self.cell.reset_recurrent_dropout_mask()
    return super(ECNN, self).call(
        inputs, mask=mask, training=training, initial_state=initial_state)

  @property
  def units(self):
    return self.cell.units

  @property
  def activation(self):
    return self.cell.activation

  @property
  def use_bias(self):
    return self.cell.use_bias

  @property
  def kernel_initializer(self):
    return self.cell.kernel_initializer

  @property
  def recurrent_initializer(self):
    return self.cell.recurrent_initializer

  @property
  def bias_initializer(self):
    return self.cell.bias_initializer

  @property
  def kernel_regularizer(self):
    return self.cell.kernel_regularizer

  @property
  def recurrent_regularizer(self):
    return self.cell.recurrent_regularizer

  @property
  def bias_regularizer(self):
    return self.cell.bias_regularizer

  @property
  def kernel_constraint(self):
    return self.cell.kernel_constraint

  @property
  def recurrent_constraint(self):
    return self.cell.recurrent_constraint

  @property
  def bias_constraint(self):
    return self.cell.bias_constraint

  @property
  def dropout(self):
    return self.cell.dropout

  @property
  def recurrent_dropout(self):
    return self.cell.recurrent_dropout

  def get_config(self):
    config = {
        'units':
            self.units,
        'activation':
            activations.serialize(self.activation),
        'use_bias':
            self.use_bias,
        'kernel_initializer':
            initializers.serialize(self.kernel_initializer),
        'recurrent_initializer':
            initializers.serialize(self.recurrent_initializer),
        'bias_initializer':
            initializers.serialize(self.bias_initializer),
        'kernel_regularizer':
            regularizers.serialize(self.kernel_regularizer),
        'recurrent_regularizer':
            regularizers.serialize(self.recurrent_regularizer),
        'bias_regularizer':
            regularizers.serialize(self.bias_regularizer),
        'activity_regularizer':
            regularizers.serialize(self.activity_regularizer),
        'kernel_constraint':
            constraints.serialize(self.kernel_constraint),
        'recurrent_constraint':
            constraints.serialize(self.recurrent_constraint),
        'bias_constraint':
            constraints.serialize(self.bias_constraint),
        'dropout':
            self.dropout,
        'recurrent_dropout':
            self.recurrent_dropout
    }
    base_config = super(ECNN, self).get_config()
    del base_config['cell']
    return dict(list(base_config.items()) + list(config.items()))

  @classmethod
  def from_config(cls, config):
    if 'implementation' in config:
      config.pop('implementation')
    return cls(**config)



# ECNN implementation example

## Preamble

In [7]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn import metrics
from sklearn.metrics import mean_squared_error
import pandas as pd
from matplotlib import pylab as plt
from sklearn.metrics import mean_absolute_error, mean_squared_error,r2_score

In [8]:
from tensorflow.keras.layers import Input,Dense
from tensorflow.keras.models import Model

In [9]:
from tensorflow.keras.layers import LSTM

## Hyper-parameter setting

In [10]:
Time_step  = 7
batch_size = 512
epochs     = 100
neurons = 32
Features   = 6

## Model

In [13]:
def my_model_1(neurons,bs,Time_step,epochs,Features=6):
    x1 = Input(batch_shape = (None,Time_step,Features))
    x2 = ECNN(neurons)(x1)
    x3 = Dense(1,activation='linear')(x2)
    return Model(x1,x3)

In [ ]:
model = my_model_1(neurons,batch_size,Time_step,epochs)

In [15]:
model.summary()

## Data pre-processing

In [17]:
import pandas as pd

In [16]:
def ScaleData(df,normalise=True):
    if normalise:
        scaler = MinMaxScaler()
        for i in list(df.columns.values):
            df[i] = scaler.fit_transform(df[i].values.reshape(-1, 1))
    return df, scaler

In [18]:
data = "path-of-datafile"
dataP = pd.read_csv(data)
data_new = dataP.drop(["Date",'Volume'],axis=1) # Removing the date colums
data_new = data_new.dropna()
Data, scaler = ScaleData(data_new)

In [19]:
Data["Observed"]= Data["Close"]

In [ ]:
values = Data.values
values.shape
print(values[:,3])

In [ ]:
values[:,-1].shape

In [22]:
def SequenceToData(df, TimeStep):
    """Data formating into (nb samples, time step, features+1)
    where +1 is the observed value"""
    X,Y = [],[]
    for idx in range(len(df)):
        end_idx = idx+TimeStep
        if end_idx > len(df)-1:
            break
        seq_X, seq_Y = df[idx:end_idx], df[:,3][end_idx]
        X.append(seq_X)
        Y.append(seq_Y)
    return np.array(X), np.array(Y)

In [23]:
# D is the data and L are the true values to be estimated
D,L = SequenceToData(values,Time_step)

In [ ]:
# checking the shape of the data. It has to be the length of the sequence-time_step
D.shape

In [25]:
#def Batches(data,bc_sz,p):
#    
#    if len(data)< bc_sz:nb_bc_sz = 1
#    else: nb_bc_sz = int(len(data)/bc_sz)
#    Data = data[:nb_bc_sz*bc_sz]
#    Trn_sz = int(nb_bc_sz*(1-p*2))*bc_sz
#    Val_sz = int(nb_bc_sz*p)*bc_sz
#    Tes_sz = int(len(Data)-(Trn_sz+Val_sz))
#    
#    return Data[:Trn_sz],Data[Trn_sz:Trn_sz+Val_sz],Data[Trn_sz+Val_sz:]

In [26]:
# x_train,x_valid,x_test = Batches(D,batch_size,0.1)
# y_train,y_valid,y_test = Batches(L,batch_size,0.1)

In [27]:
x_train, y_train = D[:5000],L[:5000]
x_valid, y_valid = D[5000:6000], L[5000:6000]
x_test, y_test   = D[6000:], L[6000:]

In [ ]:
print(y_train.shape,y_valid.shape,y_test.shape)

## Comparison to a basel model

In [29]:
# Baseline model
Real = y_test[1:]       ## from the second up to the last
Predicted = y_test[:-1] ## from the first up to the second but last

In [ ]:
plt.figure(figsize=(15, 7))
plt.plot(Real, color="blue", label="B: real ",linestyle='--', dashes=(5, 1),linewidth=2.5)
plt.plot(Predicted, color="red", label="B: predicted ");


In [ ]:
print("MSE = ",np.sqrt(mean_squared_error(Real,Predicted)))
print("r2 = ", r2_score(Real, Predicted))
print("MAE",mean_absolute_error(Real,Predicted))

## Running model

In [32]:
from tensorflow.keras.optimizers import Adam

In [33]:
model.compile(loss='mse', optimizer='adam')

    NOTE: It is not possible to save the model when eager execution is enabled. Since by default is enable in tensorflow 2.0.0-beta, the workaround is to desable it

In [34]:
from tensorflow.python.keras.callbacks import TensorBoard

In [35]:
callback = [tensorflow.keras.callbacks.ModelCheckpoint("mymodel.h5",monitor='val_loss',save_best_only=True,save_weights_only=True)]

In [ ]:
history = model.fit(x_train, y_train, epochs=epochs, batch_size=batch_size, validation_data=(x_test, y_test), verbose=1, shuffle=False,
                   callbacks=callback)
model.load_weights('mymodel.h5')
print("validation loss = ", model.evaluate(x_test,y_test,verbose=1,batch_size = batch_size))

In [ ]:
fig = plt.figure(figsize=(15,5))
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.show()

In [38]:
pred_test1 = model.predict(x_test, batch_size=batch_size)

In [39]:
y_test=np.reshape(y_test, (y_test.shape[0],1))

In [ ]:
plt.figure(figsize=(15, 5));
plt.plot(y_test, color="black", label="Actual Value")
plt.plot(pred_test1, color="green", label="Predicted Value")
plt.title('Predicted vs Actual on testig set')
plt.xlabel('Days')
plt.ylabel('Normalized Closing Price')
plt.legend(loc='best');
plt.legend(fontsize=20) # using a size in points
plt.legend(fontsize="x-large") # using a named size
#plt.grid()
plt.savefig("fi_NA_RW.pdf")
plt.show()

In [ ]:
pred_test = scaler.inverse_transform(pred_test1)
actual =scaler.inverse_transform(y_test)

actual[:5]

In [ ]:
plt.figure(figsize=(40, 10));
#plt.figure(figsize=(15, 5));
plt.plot(actual, color="black", label="Real Price",linewidth=2.5)
plt.plot(pred_test, color="green", label="ECNN without Exponential Smoothing")
#plt.title('future stock prices')
#plt.xlabel('Days')
plt.xlabel('time[days]',fontsize="large")
plt.ylabel('Closing price', fontsize="large")
plt.legend(loc='best');
plt.legend(fontsize=20) # using a size in points
plt.legend(fontsize="x-large") # using a named size
#plt.grid()
#plt.grid(color='k', linestyle='--', linewidth=0.1)
plt.savefig("ECNN without Exponential Smoothing.pdf")
plt.show()

In [ ]:
actual[-1]

In [ ]:
score = np.sqrt(mean_squared_error(actual, pred_test))
print('Test RMSE: %.4f' % (score))

In [ ]:
mean_absolute_error(actual, pred_test)

In [ ]:
r2_score(actual, pred_test)

In [ ]:
def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

MAPE = mean_absolute_percentage_error(y_test, pred_test1)

print(MAPE)